In [1]:
import urllib3
import json
import base64
import re

In [26]:
accessKey = "8972d09b-9538-44a2-bf3d-f51c50e2edf0"
imageFilePath = "sample\walkingWithDog3.png"
type = "jpg"

In [27]:
objectDetectionApiURL = "http://aiopen.etri.re.kr:8000/ObjectDetect"
humanStatusApiURL = "http://aiopen.etri.re.kr:8000/HumanStatus"
WiseNLUApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"

In [28]:
object_to_korean = {
    "person": "사람",
    "bicycle": "자전거",
    "car": "자동차",
    "motorcycle": "오토바이",
    "airplane": "비행기",
    "bus": "버스",
    "train": "기차",
    "truck": "트럭",
    "boat": "보트",
    "traffic light": "신호등",
    "fire hydrant": "소화전",
    "stop sign": "정지 신호",
    "parking meter": "주차 미터",
    "bench": "벤치",
    "bird": "새",
    "cat": "고양이",
    "dog": "개",
    "horse": "말",
    "sheep": "양",
    "cow": "소",
    "elephant": "코끼리",
    "bear": "곰",
    "zebra": "얼룩말",
    "giraffe": "기린",
    "backpack": "배낭",
    "umbrella": "우산",
    "handbag": "핸드백",
    "tie": "넥타이",
    "suitcase": "여행 가방",
    "frisbee": "프리스비",
    "skis": "스키",
    "snowboard": "스노보드",
    "sports ball": "스포츠 공",
    "kite": "연",
    "baseball bat": "야구 배트",
    "baseball glove": "야구 글러브",
    "skateboard": "스케이트보드",
    "surfboard": "서핑보드",
    "tennis racket": "테니스 라켓",
    "bottle": "병",
    "wine glass": "와인 잔",
    "cup": "컵",
    "fork": "포크",
    "knife": "칼",
    "spoon": "숟가락",
    "bowl": "그릇",
    "banana": "바나나",
    "apple": "사과",
    "sandwich": "샌드위치",
    "orange": "오렌지",
    "broccoli": "브로콜리",
    "carrot": "당근",
    "hot dog": "핫도그",
    "pizza": "피자",
    "donut": "도넛",
    "cake": "케이크",
    "chair": "의자",
    "couch": "소파",
    "potted plant": "화분",
    "bed": "침대",
    "dining table": "식탁",
    "toilet": "화장실",
    "tv": "텔레비전",
    "laptop": "노트북",
    "mouse": "마우스",
    "remote": "리모컨",
    "keyboard": "키보드",
    "cell phone": "휴대전화",
    "microwave": "전자레인지",
    "oven": "오븐",
    "toaster": "토스터",
    "sink": "싱크",
    "refrigerator": "냉장고",
    "book": "책",
    "clock": "시계",
    "vase": "꽃병",
    "scissors": "가위",
    "teddy bear": "테디 베어",
    "hair dryer": "헤어 드라이어",
    "toothbrush": "칫솔"
}

In [29]:
file = open(imageFilePath, "rb")
imageContents = base64.b64encode(file.read()).decode("utf8")
file.close()

In [30]:
imageRequestJson = { 
    "argument": {
        "type": type,
        "file": imageContents
    }
}

In [7]:
http = urllib3.PoolManager()

In [10]:
obejctDetectionResponse = http.request(
    "POST",
    objectDetectionApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8" , "Authorization": accessKey},
    body=json.dumps(imageRequestJson)
)

In [11]:
print("[responseCode] " + str(obejctDetectionResponse.status))
print("[responBody]")
print(str(obejctDetectionResponse.data, 'utf-8'))

[responseCode] 200
[responBody]
{"result":0,"return_object":{"data":[{"class":"bench","confidence":"0.987301","x":"17","y":"207","width":"486","height":"161"},{"class":"bench","confidence":"0.708714","x":"34","y":"39","width":"426","height":"336"},{"class":"cat","confidence":"0.950230","x":"52","y":"6","width":"393","height":"348"}]}}


In [12]:
obejctDetectionJson = json.loads(str(obejctDetectionResponse.data, 'utf-8'))

In [18]:
object_infos = obejctDetectionJson['return_object']['data']

objects = []
for object in object_infos:
    objects.append(object_to_korean[object['class']])

In [19]:
objects

['벤치', '벤치', '고양이']

In [20]:
status_to_korean = {
    "Standing": "서있는",
    "Walking": "걷는",
    "Crouch": "웅크리는",
    "Running": "달리는",
    "Lying": "누워 있는",
    "Sitting": "앉아 있는"
}

In [31]:
humanStatusResponse = http.request(
    "POST",
    humanStatusApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8","Authorization": accessKey},
    body=json.dumps(imageRequestJson)
)

In [23]:
str(humanStatusResponse.data, 'utf-8')

'{"result":0,"return_object":[{"data":["/home/deepview/ai-human-status-detect/src/../upload/0e588fc8-d9bf-48a1-8b55-e0ba365edc87/0.png"]}]}'

In [22]:
analysisCode = "morp"
text = "벤치에 앉아 새를 보고 있는 사람을 보여줘"

In [26]:
wordRequestJson = {  
    "argument": {
        "text": text,
        "analysis_code": analysisCode
    }
}
    
wiseNLUResponse = http.request(
    "POST",
    WiseNLUApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
    body=json.dumps(wordRequestJson)
)

In [27]:
morp_pattern = re.compile('\"morp\":\[(.*?)\]')
res = re.findall(morp_pattern, str(wiseNLUResponse.data,"utf-8"))[0]

nng_pattern = re.compile("{\"id\":\d+,\"lemma\":\"[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]*\",\"type\":\"NNG\",\"position\":\d+,\"weight\":\d.\d+}")
nng_res = re.findall(nng_pattern, res)

nng_list = []
for nng in nng_res:
    nng_dict = json.loads(nng)
    nng_list.append(nng_dict)

lemmas = [nng['lemma'] for nng in nng_list]

print(lemmas)

['벤치', '새', '사람']
